In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-alta/training.json
/kaggle/input/test-data/test_data.json


In [4]:
!nvidia-smi

Thu Jun  6 13:55:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0              28W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, LSTM, GRU,SimpleRNN, Activation, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

2024-06-06 13:55:11.078181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 13:55:11.078327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 13:55:11.195456: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import requests
import zipfile
import io
import os

# URL of the GloVe embeddings
url = "http://nlp.stanford.edu/data/glove.840B.300d.zip"
# Nama file zip
zip_file_name = "glove.840B.300d.zip"
# Folder where the embeddings will be saved
save_folder = "glove_embeddings"

# Create the folder if it does not exist
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Path to the zip file
zip_file_path = os.path.join(save_folder, zip_file_name)

# Check if the zip file already exists
if not os.path.exists(zip_file_path):
    # Download the zip file if it does not exist
    print("Downloading GloVe embeddings...")
    response = requests.get(url)
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)
    print("Download complete.")
else:
    print(f"File '{zip_file_path}' already exists, skipping download.")

# Extract the zip file
print("Extracting GloVe embeddings...")
with zipfile.ZipFile(zip_file_path, 'r') as z:
    z.extractall(save_folder)
print(f"Extraction complete. Files saved to {save_folder}.")


Download complete.
Extracting GloVe embeddings...
Extraction complete. Files saved to glove_embeddings.


In [7]:
df=pd.read_json("/kaggle/input/training-alta/training.json", lines=True)
df.head()

,text,label,id
0,Have you ever heard of the Crusades? A time in...,1,0
1,"The professors, who likely have nearly a decad...",1,1
2,Kemba Walker does a good job of defending Foye...,1,2
3,"Ganias' lawyer, Stanley Twardy, urged the gove...",1,3
4,The Circuit Court of Appeals of New Jersey had...,0,4


In [8]:
# Jumlah Max Word 
print(df['text'].apply(lambda x:len(str(x).split())).max())

192


In [9]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [10]:
## Preprocessing + Stopwords

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import re 
import string

# Preprocessing text + stopwords
def preprocess_text_stopwords(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s?!,\']', '', text) # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

In [12]:
## Preprocessing 

def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s?!,\']', '', text) # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    return text

In [13]:
df['prep_text'] = df['text'].apply(preprocess_text)
df['prep_sw_text'] = df['text'].apply(preprocess_text_stopwords)
df.head()

,text,label,id,prep_text,prep_sw_text
0,Have you ever heard of the Crusades? A time in...,1,0,have you ever heard of the crusades? a time in...,ever heard crusades? time christians went year...
1,"The professors, who likely have nearly a decad...",1,1,"the professors, who likely have nearly a decad...","professors, likely nearly decade education eac..."
2,Kemba Walker does a good job of defending Foye...,1,2,kemba walker does a good job of defending foye...,"kemba walker good job defending foye, better o..."
3,"Ganias' lawyer, Stanley Twardy, urged the gove...",1,3,"ganias' lawyer, stanley twardy, urged the gove...","ganias' lawyer, stanley twardy, urged governme..."
4,The Circuit Court of Appeals of New Jersey had...,0,4,the circuit court of appeals of new jersey had...,circuit court appeals new jersey jurisdiction ...


In [14]:
# Jumlah Max Word 
print(df['text'].apply(lambda x:len(str(x).split())).max())
print(df['prep_text'].apply(lambda x:len(str(x).split())).max())
print(df['prep_sw_text'].apply(lambda x:len(str(x).split())).max())

192
186
110


In [15]:
df['text'][0]

'Have you ever heard of the Crusades? A time in which Christians went on a 200 year rampage throughout Europe and on their path to Isreal in which they slaughtered innocent people in the name of your God?'

In [16]:
df['prep_text'][0]

'have you ever heard of the crusades? a time in which christians went on a  year rampage throughout europe and on their path to isreal in which they slaughtered innocent people in the name of your god?'

In [17]:
df['prep_sw_text'][0]

'ever heard crusades? time christians went year rampage throughout europe path isreal slaughtered innocent people name god?'

# Eksperimen 1 
* LSTM + GRU 
* Bi-LSTM + Bi-GRU

In [19]:
X_train_1, X_test_1, y_train, y_test = train_test_split(df.text.values, df.label.values, 
                                                  stratify=df.label.values, 
                                                  random_state=40, 
                                                  test_size=0.1, shuffle=True)
# using keras tokenizer here
token = Tokenizer(num_words=None)
max_len = 300

token.fit_on_texts(list(X_train_1))
X_train_seq_1 = token.texts_to_sequences(X_train_1)
X_test_seq_1 = token.texts_to_sequences(X_test_1)

#zero pad the sequences
X_train_pad_1 = pad_sequences(X_train_seq_1, maxlen=max_len)
X_test_pad_1 = pad_sequences(X_test_seq_1, maxlen=max_len)

word_index_1 = token.word_index

In [20]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/kaggle/working/glove_embeddings/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:39, 10014.18it/s]

Found 2196016 word vectors.


In [22]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix1 = np.zeros((len(word_index_1) + 1, 300))
for word, i in tqdm(word_index_1.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix1[i] = embedding_vector

100%|██████████| 32964/32964 [00:00<00:00, 241090.46it/s]


## Bi-GRU

In [23]:
# Model architecture
model7 = Sequential()
model7.add(Embedding(input_dim= len(word_index_1) + 1, output_dim=300, weights=[embedding_matrix1], input_length=300, trainable=False))
model7.add(Bidirectional(GRU(100)))
model7.add(Dropout(0.3))
model7.add(Dense(1, activation='sigmoid'))

# Compile the model
model7.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model7.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     9,889,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,889,500 (37.73 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,889,500 (37.73 MB)

In [24]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model7.fit(X_train_pad_1, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_1, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model7.evaluate(X_test_pad_1, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model7.predict(X_test_pad_1)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.8373 - loss: 0.3862 - val_accuracy: 0.9022 - val_loss: 0.2621
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8974 - loss: 0.2676 - val_accuracy: 0.9133 - val_loss: 0.2356
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9102 - loss: 0.2255 - val_accuracy: 0.9022 - val_loss: 0.2399
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9431 - loss: 0.1502 - val_accuracy: 0.9389 - val_loss: 0.1633
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9549 - loss: 0.1182 - val_accuracy: 0.9383 - val_loss: 0.1606
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9661 - loss: 0.0942 - val_accuracy: 0.9417 - val_loss: 0.1648
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9770 - loss: 0.0606 - val_accuracy: 0.9417 - val_loss: 0.1676
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9853 - loss: 0.0448 - 

## Bi-LSTM + Bi-GRU

In [25]:
# Model architecture
model1 = Sequential()
model1.add(Embedding(input_dim= len(word_index_1) + 1, output_dim=300, weights=[embedding_matrix1], input_length=300, trainable=False))
model1.add(Bidirectional(LSTM(100, return_sequences=True)))
model1.add(Bidirectional(GRU(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))

# Compile the model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     9,889,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,889,500 (37.73 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,889,500 (37.73 MB)

In [26]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model1.fit(X_train_pad_1, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_1, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model1.evaluate(X_test_pad_1, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model1.predict(X_test_pad_1)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 25s 83ms/step - accuracy: 0.8507 - loss: 0.3366 - val_accuracy: 0.9178 - val_loss: 0.2032
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9297 - loss: 0.1902 - val_accuracy: 0.9217 - val_loss: 0.1936
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9490 - loss: 0.1382 - val_accuracy: 0.9361 - val_loss: 0.1706
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9588 - loss: 0.1127 - val_accuracy: 0.9117 - val_loss: 0.2021
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9736 - loss: 0.0785 - val_accuracy: 0.9367 - val_loss: 0.1867
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9790 - loss: 0.0579 - val_accuracy: 0.9439 - val_loss: 0.1875
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9885 - loss: 0.0363 - val_accuracy: 0.9522 - val_loss: 0.1892
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9923 - loss: 0.0253 - 

In [ ]:
test = pd.read_json("/kaggle/input/test-data/test_data.json", lines=True)

X_test_alta_seq = token.texts_to_sequences(test.text)
X_test_alta_pad = pad_sequences(X_test_alta_seq, maxlen=max_len)
probabilities = model1.predict(X_test_alta_pad)
answers = np.round(probabilities).astype(int)

In [ ]:
answers = np.hstack(answers)
answers_df = pd.DataFrame(answers, columns=["label"])
answers_df["id"] = range(0, len(answers_df))
answers_df = answers_df[["id", "label"]]
answers_df.to_json("/kaggle/working/answer_lstm_gru.json", orient="records", lines=True)

## LSTM + GRU

In [27]:
# Model architecture
model2 = Sequential()
model2.add(Embedding(input_dim= len(word_index_1) + 1, output_dim=300, weights=[embedding_matrix1], input_length=300, trainable=False))
model2.add(LSTM(100, return_sequences=True))
model2.add(GRU(100))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation='sigmoid'))

# Compile the model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │     9,889,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,889,500 (37.73 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,889,500 (37.73 MB)

In [28]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model2.fit(X_train_pad_1, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_1, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model2.evaluate(X_test_pad_1, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model2.predict(X_test_pad_1)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8513 - loss: 0.3504 - val_accuracy: 0.9289 - val_loss: 0.1941
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9314 - loss: 0.1791 - val_accuracy: 0.9322 - val_loss: 0.1682
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9493 - loss: 0.1356 - val_accuracy: 0.9417 - val_loss: 0.1580
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9595 - loss: 0.1055 - val_accuracy: 0.9433 - val_loss: 0.1520
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9717 - loss: 0.0827 - val_accuracy: 0.9433 - val_loss: 0.1576
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9848 - loss: 0.0491 - val_accuracy: 0.9417 - val_loss: 0.1720
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9879 - loss: 0.0372 - val_accuracy: 0.9539 - val_loss: 0.1658
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9928 - loss: 0.0223 - 

## Eksperimen 2

In [29]:
df.head()

,text,label,id,prep_text,prep_sw_text
0,Have you ever heard of the Crusades? A time in...,1,0,have you ever heard of the crusades? a time in...,ever heard crusades? time christians went year...
1,"The professors, who likely have nearly a decad...",1,1,"the professors, who likely have nearly a decad...","professors, likely nearly decade education eac..."
2,Kemba Walker does a good job of defending Foye...,1,2,kemba walker does a good job of defending foye...,"kemba walker good job defending foye, better o..."
3,"Ganias' lawyer, Stanley Twardy, urged the gove...",1,3,"ganias' lawyer, stanley twardy, urged the gove...","ganias' lawyer, stanley twardy, urged governme..."
4,The Circuit Court of Appeals of New Jersey had...,0,4,the circuit court of appeals of new jersey had...,circuit court appeals new jersey jurisdiction ...


In [30]:
X_train_2, X_test_2, y_train, y_test = train_test_split(df.prep_text.values, df.label.values, 
                                                  stratify=df.label.values, 
                                                  random_state=40, 
                                                  test_size=0.1, shuffle=True)
# using keras tokenizer here
token = Tokenizer(num_words=None)
max_len = 300

token.fit_on_texts(list(X_train_2))
X_train_seq_2 = token.texts_to_sequences(X_train_2)
X_test_seq_2 = token.texts_to_sequences(X_test_2)

#zero pad the sequences
X_train_pad_2 = pad_sequences(X_train_seq_2, maxlen=max_len)
X_test_pad_2 = pad_sequences(X_test_seq_2, maxlen=max_len)

word_index_2 = token.word_index

In [31]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix2 = np.zeros((len(word_index_2) + 1, 300))
for word, i in tqdm(word_index_2.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix2[i] = embedding_vector

100%|██████████| 31440/31440 [00:00<00:00, 230625.54it/s]


## Bi-GRU

In [32]:
# Model architecture
model8 = Sequential()
model8.add(Embedding(input_dim= len(word_index_2) + 1, output_dim=300, weights=[embedding_matrix2], input_length=300, trainable=False))
model8.add(Bidirectional(GRU(100)))
model8.add(Dropout(0.3))
model8.add(Dense(1, activation='sigmoid'))

# Compile the model
model8.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model8.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │     9,432,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,432,300 (35.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,432,300 (35.98 MB)

In [33]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model8.fit(X_train_pad_2, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_2, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model8.evaluate(X_test_pad_2, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model8.predict(X_test_pad_2)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.8313 - loss: 0.3974 - val_accuracy: 0.8939 - val_loss: 0.2795
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8991 - loss: 0.2611 - val_accuracy: 0.9128 - val_loss: 0.2124
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9209 - loss: 0.1965 - val_accuracy: 0.9367 - val_loss: 0.1738
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9450 - loss: 0.1447 - val_accuracy: 0.9356 - val_loss: 0.1693
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9564 - loss: 0.1127 - val_accuracy: 0.9333 - val_loss: 0.1664
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9702 - loss: 0.0823 - val_accuracy: 0.9394 - val_loss: 0.1569
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9786 - loss: 0.0615 - val_accuracy: 0.9444 - val_loss: 0.1548
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9841 - loss: 0.0464 - 

## Bi-LSTM + Bi-GRU

In [34]:
# Model architecture
model3 = Sequential()
model3.add(Embedding(input_dim= len(word_index_2) + 1, output_dim=300, weights=[embedding_matrix2], input_length=300, trainable=False))
model3.add(Bidirectional(LSTM(100, return_sequences=True)))
model3.add(Bidirectional(GRU(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1, activation='sigmoid'))

# Compile the model
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │     9,432,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,432,300 (35.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,432,300 (35.98 MB)

In [35]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model3.fit(X_train_pad_2, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_2, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model3.evaluate(X_test_pad_2, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model3.predict(X_test_pad_2)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 24s 83ms/step - accuracy: 0.8537 - loss: 0.3446 - val_accuracy: 0.9289 - val_loss: 0.1979
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9278 - loss: 0.1885 - val_accuracy: 0.9189 - val_loss: 0.2046
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9403 - loss: 0.1523 - val_accuracy: 0.9356 - val_loss: 0.1626
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9592 - loss: 0.1109 - val_accuracy: 0.9278 - val_loss: 0.1908
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9703 - loss: 0.0844 - val_accuracy: 0.9233 - val_loss: 0.2214
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9794 - loss: 0.0575 - val_accuracy: 0.9394 - val_loss: 0.1624
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9871 - loss: 0.0392 - val_accuracy: 0.9472 - val_loss: 0.1737
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9886 - loss: 0.0337 - 

## Bi-GRU + Bi-LSTM

In [71]:
# Model architecture
model23 = Sequential()
model23.add(Embedding(input_dim= len(word_index_2) + 1, output_dim=300, weights=[embedding_matrix2], input_length=300, trainable=False))
model23.add(Bidirectional(GRU(100, return_sequences=True)))
model23.add(Bidirectional(LSTM(100)))
model23.add(Dropout(0.3))
model23.add(Dense(1, activation='sigmoid'))

# Compile the model
model23.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model23.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_22 (Embedding)        │ ?                      │     9,432,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_25                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_26                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,432,300 (35.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,432,300 (35.98 MB)

In [72]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model23.fit(X_train_pad_2, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_2, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model23.evaluate(X_test_pad_2, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model23.predict(X_test_pad_2)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 24s 82ms/step - accuracy: 0.8480 - loss: 0.3705 - val_accuracy: 0.9061 - val_loss: 0.2530
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.8991 - loss: 0.2566 - val_accuracy: 0.9233 - val_loss: 0.2040
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.9276 - loss: 0.1841 - val_accuracy: 0.9239 - val_loss: 0.2059
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.9448 - loss: 0.1409 - val_accuracy: 0.9361 - val_loss: 0.1666
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.9645 - loss: 0.0934 - val_accuracy: 0.9400 - val_loss: 0.1587
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.9733 - loss: 0.0727 - val_accuracy: 0.9383 - val_loss: 0.1834
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.9850 - loss: 0.0426 - val_accuracy: 0.9506 - val_loss: 0.1612
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.9835 - loss: 0.0458 - 

## LSTM + GRU

In [36]:
# Model architecture
model4 = Sequential()
model4.add(Embedding(input_dim= len(word_index_2) + 1, output_dim=300, weights=[embedding_matrix2], input_length=300, trainable=False))
model4.add(LSTM(100, return_sequences=True))
model4.add(GRU(100))
model4.add(Dropout(0.3))
model4.add(Dense(1, activation='sigmoid'))

# Compile the model
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │     9,432,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,432,300 (35.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,432,300 (35.98 MB)

In [37]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model4.fit(X_train_pad_2, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_2, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model4.evaluate(X_test_pad_2, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model4.predict(X_test_pad_2)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8604 - loss: 0.3415 - val_accuracy: 0.9206 - val_loss: 0.2043
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9256 - loss: 0.1953 - val_accuracy: 0.9272 - val_loss: 0.1810
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9387 - loss: 0.1599 - val_accuracy: 0.9300 - val_loss: 0.2010
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9540 - loss: 0.1289 - val_accuracy: 0.9400 - val_loss: 0.1593
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9674 - loss: 0.0874 - val_accuracy: 0.9494 - val_loss: 0.1551
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9773 - loss: 0.0643 - val_accuracy: 0.9439 - val_loss: 0.1639
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9869 - loss: 0.0426 - val_accuracy: 0.9511 - val_loss: 0.1474
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9914 - loss: 0.0264 - 

## GRU-LSTM

In [51]:
# Model architecture
model10 = Sequential()
model10.add(Embedding(input_dim= len(word_index_2) + 1, output_dim=300, weights=[embedding_matrix2], input_length=300, trainable=False))
model10.add(GRU(100, return_sequences=True))
model10.add(LSTM(100))
model10.add(Dropout(0.3))
model10.add(Dense(1, activation='sigmoid'))

# Compile the model
model10.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model10.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ ?                      │     9,432,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_12 (GRU)                    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,432,300 (35.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,432,300 (35.98 MB)

In [52]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model10.fit(X_train_pad_2, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_2, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model10.evaluate(X_test_pad_2, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model10.predict(X_test_pad_2)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8458 - loss: 0.3675 - val_accuracy: 0.9106 - val_loss: 0.2474
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9002 - loss: 0.2617 - val_accuracy: 0.9083 - val_loss: 0.2386
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9126 - loss: 0.2197 - val_accuracy: 0.9317 - val_loss: 0.1856
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9372 - loss: 0.1615 - val_accuracy: 0.9344 - val_loss: 0.1672
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9532 - loss: 0.1221 - val_accuracy: 0.9367 - val_loss: 0.1628
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9655 - loss: 0.0923 - val_accuracy: 0.9328 - val_loss: 0.1798
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9782 - loss: 0.0599 - val_accuracy: 0.9356 - val_loss: 0.1837
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9842 - loss: 0.0450 - 

## Eksperimen 3

In [38]:
df.head()

,text,label,id,prep_text,prep_sw_text
0,Have you ever heard of the Crusades? A time in...,1,0,have you ever heard of the crusades? a time in...,ever heard crusades? time christians went year...
1,"The professors, who likely have nearly a decad...",1,1,"the professors, who likely have nearly a decad...","professors, likely nearly decade education eac..."
2,Kemba Walker does a good job of defending Foye...,1,2,kemba walker does a good job of defending foye...,"kemba walker good job defending foye, better o..."
3,"Ganias' lawyer, Stanley Twardy, urged the gove...",1,3,"ganias' lawyer, stanley twardy, urged the gove...","ganias' lawyer, stanley twardy, urged governme..."
4,The Circuit Court of Appeals of New Jersey had...,0,4,the circuit court of appeals of new jersey had...,circuit court appeals new jersey jurisdiction ...


In [39]:
X_train_3, X_test_3, y_train, y_test = train_test_split(df.prep_sw_text.values, df.label.values, 
                                                  stratify=df.label.values, 
                                                  random_state=40, 
                                                  test_size=0.1, shuffle=True)
# using keras tokenizer here
token = Tokenizer(num_words=None)
max_len = 300

token.fit_on_texts(list(X_train_3))
X_train_seq_3 = token.texts_to_sequences(X_train_3)
X_test_seq_3 = token.texts_to_sequences(X_test_3)

#zero pad the sequences
X_train_pad_3 = pad_sequences(X_train_seq_3, maxlen=max_len)
X_test_pad_3 = pad_sequences(X_test_seq_3, maxlen=max_len)

word_index_3 = token.word_index

In [40]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix3 = np.zeros((len(word_index_3) + 1, 300))
for word, i in tqdm(word_index_3.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix3[i] = embedding_vector

100%|██████████| 31389/31389 [00:00<00:00, 237856.09it/s]


## Bi-GRU

In [41]:
# Model architecture
model9 = Sequential()
model9.add(Embedding(input_dim= len(word_index_3) + 1, output_dim=300, weights=[embedding_matrix3], input_length=300, trainable=False))
model9.add(Bidirectional(GRU(100)))
model9.add(Dropout(0.3))
model9.add(Dense(1, activation='sigmoid'))

# Compile the model
model9.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model9.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │     9,417,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,417,000 (35.92 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,417,000 (35.92 MB)

In [42]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model9.fit(X_train_pad_3, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_3, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model9.evaluate(X_test_pad_3, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model9.predict(X_test_pad_3)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.8333 - loss: 0.3815 - val_accuracy: 0.9061 - val_loss: 0.2511
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9014 - loss: 0.2476 - val_accuracy: 0.9289 - val_loss: 0.1972
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9354 - loss: 0.1693 - val_accuracy: 0.9361 - val_loss: 0.1803
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9434 - loss: 0.1423 - val_accuracy: 0.9194 - val_loss: 0.2068
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9586 - loss: 0.1122 - val_accuracy: 0.9389 - val_loss: 0.1672
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9697 - loss: 0.0842 - val_accuracy: 0.9411 - val_loss: 0.1692
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9761 - loss: 0.0646 - val_accuracy: 0.9217 - val_loss: 0.2197
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9863 - loss: 0.0412 - 

## Bi-LSTM + Bi-GRU

In [44]:
# Model architecture
model5 = Sequential()
model5.add(Embedding(input_dim= len(word_index_3) + 1, output_dim=300, weights=[embedding_matrix3], input_length=300, trainable=False))
model5.add(Bidirectional(LSTM(100, return_sequences=True)))
model5.add(Bidirectional(GRU(100)))
model5.add(Dropout(0.3))
model5.add(Dense(1, activation='sigmoid'))

# Compile the model
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │     9,417,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,417,000 (35.92 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,417,000 (35.92 MB)

In [45]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model5.fit(X_train_pad_3, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_3, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model5.evaluate(X_test_pad_3, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model5.predict(X_test_pad_3)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 24s 83ms/step - accuracy: 0.8734 - loss: 0.3169 - val_accuracy: 0.9356 - val_loss: 0.1831
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9352 - loss: 0.1737 - val_accuracy: 0.9367 - val_loss: 0.1786
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9504 - loss: 0.1324 - val_accuracy: 0.9317 - val_loss: 0.1897
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.9659 - loss: 0.0933 - val_accuracy: 0.9439 - val_loss: 0.1969
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9755 - loss: 0.0658 - val_accuracy: 0.9450 - val_loss: 0.1650
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9873 - loss: 0.0386 - val_accuracy: 0.9317 - val_loss: 0.2194
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9858 - loss: 0.0416 - val_accuracy: 0.9389 - val_loss: 0.2053
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9948 - loss: 0.0176 - 

## Bi-GRU + Bi-LSTM

In [59]:
# Model architecture
model32 = Sequential()
model32.add(Embedding(input_dim= len(word_index_3) + 1, output_dim=300, weights=[embedding_matrix3], input_length=300, trainable=False))
model32.add(Bidirectional(LSTM(100, return_sequences=True)))
model32.add(Bidirectional(GRU(100)))
model32.add(Dropout(0.3))
model32.add(Dense(1, activation='sigmoid'))

# Compile the model
model32.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model32.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ ?                      │     9,417,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_14                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,417,000 (35.92 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,417,000 (35.92 MB)

In [60]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model32.fit(X_train_pad_3, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_3, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model32.evaluate(X_test_pad_3, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model32.predict(X_test_pad_3)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 25s 83ms/step - accuracy: 0.8772 - loss: 0.3119 - val_accuracy: 0.9322 - val_loss: 0.1917
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9270 - loss: 0.1860 - val_accuracy: 0.9356 - val_loss: 0.1866
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9468 - loss: 0.1374 - val_accuracy: 0.9361 - val_loss: 0.1860
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9629 - loss: 0.0977 - val_accuracy: 0.9489 - val_loss: 0.1596
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.9769 - loss: 0.0651 - val_accuracy: 0.9517 - val_loss: 0.1619
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9888 - loss: 0.0375 - val_accuracy: 0.9172 - val_loss: 0.2308
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.9901 - loss: 0.0294 - val_accuracy: 0.9489 - val_loss: 0.1927
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.9887 - loss: 0.0348 - 

## LSTM + GRU

In [46]:
# Model architecture
model6 = Sequential()
model6.add(Embedding(input_dim= len(word_index_3) + 1, output_dim=300, weights=[embedding_matrix3], input_length=300, trainable=False))
model6.add(LSTM(100, return_sequences=True))
model6.add(GRU(100))
model6.add(Dropout(0.3))
model6.add(Dense(1, activation='sigmoid'))

# Compile the model
model6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │     9,417,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,417,000 (35.92 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,417,000 (35.92 MB)

In [47]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model6.fit(X_train_pad_3, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_3, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model6.evaluate(X_test_pad_3, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model6.predict(X_test_pad_3)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8678 - loss: 0.3152 - val_accuracy: 0.9289 - val_loss: 0.1916
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9295 - loss: 0.1848 - val_accuracy: 0.9211 - val_loss: 0.2027
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9458 - loss: 0.1426 - val_accuracy: 0.9356 - val_loss: 0.1811
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9632 - loss: 0.1023 - val_accuracy: 0.9344 - val_loss: 0.1827
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9773 - loss: 0.0653 - val_accuracy: 0.9356 - val_loss: 0.1788
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9488 - loss: 0.1488 - val_accuracy: 0.9367 - val_loss: 0.1686
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9772 - loss: 0.0636 - val_accuracy: 0.9400 - val_loss: 0.1743
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9930 - loss: 0.0239 - 

## GRU + LSTM

In [57]:
# Model architecture
model30 = Sequential()
model30.add(Embedding(input_dim= len(word_index_3) + 1, output_dim=300, weights=[embedding_matrix3], input_length=300, trainable=False))
model30.add(GRU(100, return_sequences=True))
model30.add(LSTM(100))
model30.add(Dropout(0.3))
model30.add(Dense(1, activation='sigmoid'))

# Compile the model
model30.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model30.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ ?                      │     9,417,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_15 (GRU)                    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,417,000 (35.92 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,417,000 (35.92 MB)

In [58]:
# Train the model
batch_size = 64
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

with tf.device("/GPU:0"):
    model30.fit(X_train_pad_3, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad_3, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model30.evaluate(X_test_pad_3, y_test)
print(f"Test Accuracy: {accuracy}")

scores = model30.predict(X_test_pad_3)
print("Auc: %.4f%%" % (roc_auc(scores,y_test)))

Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8649 - loss: 0.3422 - val_accuracy: 0.9261 - val_loss: 0.2098
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9331 - loss: 0.1848 - val_accuracy: 0.9311 - val_loss: 0.1851
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9406 - loss: 0.1533 - val_accuracy: 0.9289 - val_loss: 0.1846
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9464 - loss: 0.1327 - val_accuracy: 0.9361 - val_loss: 0.1730
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9616 - loss: 0.1029 - val_accuracy: 0.9344 - val_loss: 0.1818
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9754 - loss: 0.0712 - val_accuracy: 0.9400 - val_loss: 0.1779
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9833 - loss: 0.0494 - val_accuracy: 0.9406 - val_loss: 0.1783
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9918 - loss: 0.0293 - 